<a href="https://colab.research.google.com/github/shin1038/shin1038/blob/main/Mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# CNN 설계하기
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping

import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

# seed값 설정
seed=0
np.random.seed(seed)
tf.random.set_seed(3)

In [ ]:
# 데이터 불러오기
(X_train, Y_train),(X_test, Y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], 28, 28,1).astype('float32') / 255
X_test = X_test.reshape(X_test.shape[0], 28, 28,1).astype('float32') / 255

Y_train = np_utils.to_categorical(Y_train)
Y_test = np_utils.to_categorical(Y_test)

In [ ]:
model=Sequential() 

model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu", input_shape=(28,28,1)))
model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=128, kernel_size = (3,3), activation="relu"))
model.add(Conv2D(filters=128, kernel_size = (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))   
model.add(Conv2D(filters=256, kernel_size = (3,3), activation="relu"))    
model.add(MaxPooling2D(pool_size=(2,2)))    
model.add(Flatten())
model.add(Dense(512,activation="relu"))    
model.add(Dense(10,activation="softmax"))   

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# 모델 최적화 설정
CNNMODEL_DIR = './CNNmodel/'
if not os.path.exists(CNNMODEL_DIR):
    os.mkdir(CNNMODEL_DIR)
    
modelpath='./CNNmodel/{epoch:02d}-{val_loss:.4f}.hdf5'
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10)

In [ ]:
# 모델의 실행
history= model.fit(X_train,Y_train, validation_data=(X_test,Y_test),epochs=10, verbose=0, batch_size=50, callbacks=[early_stopping_callback, checkpointer])


In [ ]:
# 테스트 정확도 출력
print('\n Test Accuray: %.4f' % (model.evaluate(X_test,Y_test)[1]))

In [ ]:
# 테스트셋의 오차
y_vloss = history.history['val_loss']

#학습셋의 오차
y_loss = history.history['loss']

# 그래프로 표현
x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c='red', label='Testset_loss')
plt.plot(x_len, y_loss, marker='.', c='blue', label='Trainset_loss')

# 그래프에 그리드를 주고 레이블을 표시

plt.legend(loc = 'upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()